---
## Module Import
---

In [1]:
import pandas as pd
import os
import gensim
import numpy as np
import time
import pickle
import warnings
import matplotlib.pyplot as plt
from imblearn.over_sampling import RandomOverSampler
from gensim.models.fasttext import FastText as FT_gensim
from datetime import datetime
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout, Conv1D, GlobalMaxPool1D, SimpleRNN
from keras import backend as K
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import IsolationForest
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

Using TensorFlow backend.


In [2]:
import pandas as pd
#transport = pd.read_csv('transport_classifier_train_20191212.csv')

before1hour = 'T_AD_XDR_TR_KNOWN_' + '20200121'

conn = pymysql.connect(host ='192.168.6.89' , port = 3306,
                       user = 'atom', password = 'atom', db = '5G_Probe_App_Discovery', charset = 'utf8',
                      cursorclass = pymysql.cursors.DictCursor)
# 커서 가져오기
curs = conn.cursor()
sql = "SELECT * FROM " + before1hour + " WHERE OngoingFlag = 1 AND SummaryCreateTime >= unix_timestamp(date_add('2020-01-21 10:00', interval -1 hour)) AND SummaryCreateTime < unix_timestamp('2020-01-21 10-00');"
curs.execute(sql)
# 데이터 가져오기
transport = pd.DataFrame(curs.fetchall())
transport = transport.dropna(subset = ['App_Group_Code']).reset_index(drop=True)
transport = transport[['SummaryCreateTime', 'IMSI', 'Gender', 'Age', 'App_ID', 'App_Group_Code', 'App_Port','Protocol', 'IP_Version', 'App_IPv4', 'App_IPv6', 'DNS.QueryName', 'DNS.LastDnsTimeToLive', 'Traffic.Delta.Dnlink.DataSize', 'Traffic.Delta.Uplink.DataSize', 'TCP.Delta.MSS.SynAckMssSize', 'Traffic.Delta.Uplink.RetransPayloadSize', 'Traffic.Delta.Dnlink.RetransPayloadSize']].copy()

transport.head()

---
#### 데이터 만들때는 app_id 개수가 5개이하인건 제외하고 생각
#### 이상치 알고리즘에서 6개이상인것만 신경쓰도록 해야한다. 5개이하인 app_id 개수 : 156개
#### 6개이상인 appid에 대해서 2개씩을 이상치로 놓음
---

In [3]:
transport['new_label'] = transport['app_id']

In [4]:
transport.groupby('app_id').count()['index'].sort_values()[-20: ]

app_id
F023     12909
H017     13098
C5F5     13168
EE9F     13419
EAD5     13514
F021     14378
H014     15857
EACA     16363
EAE6     16594
EACC     16746
H012     17984
EC43     20566
EAA9     33956
EB16     38881
H032     39324
EAF0     41767
ED08     51130
H020     52915
C446     79965
EAF8    166843
Name: index, dtype: int64

In [5]:
appid_6 = list(transport.groupby('app_id').count()['index'].sort_values()[-20: ].index)

In [6]:
for i in range(len(appid_6)):
    if i < len(appid_6)-1:
        transport.loc[list(transport[transport.app_id == appid_6[i]].iloc[-4000 : ].index)[:], 'new_label'] = appid_6[i+1]
        transport.loc[list(transport[transport.app_id == appid_6[i]].iloc[ : -4000].index)[:], 'new_label'] = appid_6[i]
    else : 
        transport.loc[list(transport[transport.app_id == appid_6[i]].iloc[-4000 : ].index)[:], 'new_label'] = appid_6[0] 
        transport.loc[list(transport[transport.app_id == appid_6[i]].iloc[ : -4000].index)[:], 'new_label'] = appid_6[i]  

---
## DNS Vector화
---

In [7]:
from DNS_Encoding.data_nodns_del import data_nodns_del
from DNS_Encoding.make_fasttext_model_vector2 import make_fasttext_model_vector2
from DNS_Encoding.making_training_bydns import making_training_bydns
transport_1 = data_nodns_del(transport)
transport_vector,transport_modelinit, max_length_transport = make_fasttext_model_vector2(transport_1)
transportdns_training = making_training_bydns(transport,transport_vector)

---
## IP Vector화
---

In [8]:
from IP_Encoding.ipencoding_65536_delH import ipencoding_65536_delH

transport_ip65536_delH = ipencoding_65536_delH(transport)

---
## ETC Features Vecotr화 
---

In [9]:
from Etc_features.age_onehot import age_onehot
from Etc_features.gender_onehot import gender_onehot
from Etc_features.port_onehot import port_onehot
from Etc_features.protocol_onehot import protocol_onehot
from Etc_features.time_hour_onehot import time_hour_onehot
from Etc_features.train_delta_dn_link_data_size import train_delta_dn_link_data_size
from Etc_features.train_delta_up_link_data_size import train_delta_up_link_data_size
from Etc_features.train_lastdnstimetolive import train_lastdnstimetolive
from Etc_features.train_synackmsssize import train_synackmsssize
def train_delta_dn_link_payload_size(data):
    min_max_scaler = MinMaxScaler()
    fitted = min_max_scaler.fit(data[['delta_dn_link_payload_size']].astype('float64'))
    output = min_max_scaler.transform(data[['delta_dn_link_payload_size']].astype('float64'))
    output = pd.DataFrame(output)
    output.columns = ['delta_dn_link_payload_size']

    #추가(20191023), 최종 모델링에서는 삭제해야함
    output.index = data.index
    return pd.DataFrame(output)

def train_delta_up_link_payload_size(data):
    min_max_scaler = MinMaxScaler()
    fitted = min_max_scaler.fit(data[['delta_up_link_payload_size']].astype('float64'))
    output = min_max_scaler.transform(data[['delta_up_link_payload_size']].astype('float64'))
    output = pd.DataFrame(output)
    output.columns = ['delta_up_link_payload_size']

    #추가(20191023), 최종 모델링에서는 삭제해야함
    output.index = data.index
    return pd.DataFrame(output)

transport_age = age_onehot(transport)
transport_gender = gender_onehot(transport)
transport_port = port_onehot(transport)
transport_protocol = protocol_onehot(transport)
transport_timehour = time_hour_onehot(transport)
transport_lastdnstimetolive = train_lastdnstimetolive(transport)
transport_delta_dn_link_data_size = train_delta_dn_link_data_size(transport)
transport_delta_up_link_data_size = train_delta_up_link_data_size(transport)
transport_synackmsssize = train_synackmsssize(transport)
transport_train_delta_dn_link_payload_size = train_delta_dn_link_payload_size(transport)
transport_train_delta_up_link_payload_size = train_delta_up_link_payload_size(transport)


transport_training_ip65536_delH = pd.concat([transport_ip65536_delH, transportdns_training, transport_age, transport_gender, transport_port, 
                                        transport_protocol, transport_timehour, transport_lastdnstimetolive,transport_delta_dn_link_data_size,transport_delta_up_link_data_size,
                                        transport_synackmsssize, transport_train_delta_up_link_payload_size, transport_train_delta_dn_link_payload_size], axis = 1)


/data01/CSB/CSB_Jupyter/PROBE/DEMO/Etc_features/port_onehot.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  b['app_port'] = 'x'
/data01/CSB/CSB_Jupyter/PROBE/DEMO/Etc_features/protocol_onehot.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  b['protocol'] = 'x'


In [10]:
app_id = list(transport.groupby('app_id').count().index)
transport_training_ip65536_delH = pd.concat([transport_training_ip65536_delH , transport[['app_id']]], axis=1)
transport_training_ip65536_delH = pd.concat([transport_training_ip65536_delH , transport[['new_label']]], axis=1)

---
#### isolation forest 1, -1
---

In [ ]:
a = pd.DataFrame(columns = http_training_ip65536_delH.columns)


for i in range(len(app_id)):
    if len(http_training_ip65536_delH[http_training_ip65536_delH.app_id == app_id[i]])<4:
        xx = http_training_ip65536_delH.loc[list(http_training_ip65536_delH[http_training_ip65536_delH.app_id == app_id[i]].index)]
        xx['anomaly_score'] = [2 for i in range(len(xx))]
        a = pd.concat([a,xx])

        
###### 2는 3개이하짜리들 
    
    else:
        clf = IsolationForest(n_estimators=50, contamination=0.10, random_state=35, verbose=0)
        y_pred = clf.fit_predict(http_training_ip65536_delH[http_training_ip65536_delH.app_id == app_id[i]].drop(['app_id','new_label'], axis = 1)) #1,-1로 나온다.
        
        xx = http_training_ip65536_delH.loc[list(http_training_ip65536_delH[http_training_ip65536_delH.app_id == app_id[i]].index)]
        xx['anomaly_score'] = list(y_pred)
        a = pd.concat([a,xx])
        
        
###### -1이 이상치, 1이 정상.

In [ ]:
x = sum(a[a.app_id != a.new_label].anomaly_score == -1)
y = sum(a[a.app_id != a.new_label].anomaly_score == 1)
print('이상치 중에서 실제로 이상치로 모델이 판별한 비율 : ' + str(x/(x+y)*100))

In [ ]:
z = sum(a[a.app_id == a.new_label].anomaly_score == -1)
w = sum(a[a.app_id == a.new_label].anomaly_score == 1)
print('정상 중에서 이상치로 잘못 판별한 비율 : ' + str(z/(z+w)*100))

---
#### isolation forest score 값
---

In [11]:
a = pd.DataFrame(columns = transport_training_ip65536_delH.columns)

for i in range(len(app_id)):
    if len(transport_training_ip65536_delH[transport_training_ip65536_delH.app_id == app_id[i]])<12909:
        xx = transport_training_ip65536_delH.loc[list(transport_training_ip65536_delH[transport_training_ip65536_delH.app_id == app_id[i]].index)]
        xx['anomaly_score'] = [2 for i in range(len(xx))]
        a = pd.concat([a,xx])

            
    else:
        clf = IsolationForest(n_estimators=50, contamination=0.30, random_state=35, verbose=0)
        clf.fit(transport_training_ip65536_delH[transport_training_ip65536_delH.app_id == app_id[i]].drop(['app_id','new_label'], axis = 1))
        y_pred = clf.score_samples(transport_training_ip65536_delH[transport_training_ip65536_delH.app_id == app_id[i]].drop(['app_id','new_label'], axis = 1)) #1,-1로 나온다.
        xx = transport_training_ip65536_delH.loc[list(transport_training_ip65536_delH[transport_training_ip65536_delH.app_id == app_id[i]].index)]
        xx['anomaly_score'] = list(y_pred)
        a = pd.concat([a,xx])
        
        
###### -1이 이상치, 1이 정상.

/home/chkimsu/anaconda3/envs/csb2/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys
/home/chkimsu/anaconda3/envs/csb2/lib/python3.6/site-packages/sklearn/ensemble/iforest.py:247: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)
/home/chkimsu/anaconda3/envs/csb2/lib/python3.6/site-packages/sklearn/ensemble/iforest.py:247: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)
/home/chkimsu/anac

In [13]:
print('전체 데이터 개수 : '+str(len(a)))
print('모델이 이상치로 판별한 데이터 개수 : ' + str(len(a[a.anomaly_score<-0.45])))
print('데이터 실제 이상치 개수 : ' + str(sum(a.app_id != a.new_label)))
print('이상치로 판별한 데이터 중 실제 이상치 개수 : '+ str(sum(a[a.anomaly_score<-0.45].app_id != a[a.anomaly_score<-0.45].new_label)))

전체 데이터 개수 : 1065997
모델이 이상치로 판별한 데이터 개수 : 85956
데이터 실제 이상치 개수 : 80000
이상치로 판별한 데이터 중 실제 이상치 개수 : 13002


---
#### 권대리님 모델에 태울 것
---

In [ ]:
http.loc[list(a[a.anomaly_score <-0.5].index)].to_csv('modelinputhttp.csv')